In [10]:
import uproot
import glob
import json
import pandas as pd
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
import numpy as np
import itertools

from utils.analysis import PandasAnalysis
from utils.cutflow import Cutflow

TAG = "btagsf_fix"
# TAG = "jec_fix"
BLIND = False

In [11]:
babies = glob.glob(f"../analysis/studies/vbswh/output_{TAG}/Run2/*.root")
babies = [baby for baby in babies if "Lambda" not in baby]
data_babies = [baby for baby in babies if "data" in baby]
sig_babies = [baby for baby in babies if "VBSWH_mkW" in baby]
bkg_babies = [baby for baby in babies if not "data" in baby and not "VBS" in baby]
print(data_babies)
print(sig_babies)
bkg_babies

['../analysis/studies/vbswh/output_btagsf_fix/Run2/data.root']
['../analysis/studies/vbswh/output_btagsf_fix/Run2/VBSWH_mkW.root']


['../analysis/studies/vbswh/output_btagsf_fix/Run2/TTbar2L.root',
 '../analysis/studies/vbswh/output_btagsf_fix/Run2/WJets.root',
 '../analysis/studies/vbswh/output_btagsf_fix/Run2/TTbar1L.root',
 '../analysis/studies/vbswh/output_btagsf_fix/Run2/SingleTop.root',
 '../analysis/studies/vbswh/output_btagsf_fix/Run2/Bosons.root',
 '../analysis/studies/vbswh/output_btagsf_fix/Run2/EWKWLep.root',
 '../analysis/studies/vbswh/output_btagsf_fix/Run2/VH.root',
 '../analysis/studies/vbswh/output_btagsf_fix/Run2/TTX.root',
 '../analysis/studies/vbswh/output_btagsf_fix/Run2/VVJJ.root']

In [12]:
vbswh = PandasAnalysis(
    sig_root_files=sig_babies,
    bkg_root_files=[b for b in bkg_babies if "VBSWH_SM" not in b],
    data_root_files=data_babies,
    ttree_name="tree",
    weight_columns=[
        "xsec_sf", "ewkfix_sf", 
        "elec_id_sf", "muon_id_sf", 
        "elec_reco_sf", "muon_iso_sf", 
        "light_btag_sf", "bc_btag_sf",
        "pu_sf", "prefire_sf", "puid_sf",
        "trig_elec_sf", "trig_muon_sf", 
        "xbb_sf" # applied only because Xbb > 0.9 applied everywhere for SR1 and SR2
    ],
    reweight_column="reweights"
)
vbswh.df["unity"] = 1 # IMPORTANT
vbswh.df["presel_noDetaJJ"] = vbswh.df.eval(
    "passes_bveto and M_jj > 500 and hbbjet_score > 0.3"
)
vbswh.df["presel"] = vbswh.df.eval(
    "passes_bveto and M_jj > 500 and abs(deta_jj) > 3 and hbbjet_score > 0.3"
)
vbswh.df["SR2"] = vbswh.df.eval(
    "presel_noDetaJJ and M_jj > 600 and ST > 1500 and hbbjet_score > 0.9 and abs(deta_jj) > 4 and hbbjet_msoftdrop < 150"
)
vbswh.df["SR1"] = vbswh.df.eval(
    "presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and abs(deta_jj) > 4 and hbbjet_msoftdrop < 150"
)
vbswh.df["SR2_up"] = vbswh.df.eval(
    "presel_noDetaJJ and M_jj > 600 and ST_up > 1500 and hbbjet_score > 0.9 and abs(deta_jj) > 4 and hbbjet_msoftdrop < 150"
)
vbswh.df["SR1_up"] = vbswh.df.eval(
    "presel_noDetaJJ and M_jj > 600 and ST_up > 900 and hbbjet_score > 0.9 and abs(deta_jj) > 4 and hbbjet_msoftdrop < 150"
)
vbswh.df["SR2_dn"] = vbswh.df.eval(
    "presel_noDetaJJ and M_jj > 600 and ST_dn > 1500 and hbbjet_score > 0.9 and abs(deta_jj) > 4 and hbbjet_msoftdrop < 150"
)
vbswh.df["SR1_dn"] = vbswh.df.eval(
    "presel_noDetaJJ and M_jj > 600 and ST_dn > 900 and hbbjet_score > 0.9 and abs(deta_jj) > 4 and hbbjet_msoftdrop < 150"
)
vbswh.df["regionA"] = vbswh.df.eval(
    "presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and abs(deta_jj) > 4 and hbbjet_msoftdrop >= 150"
)
vbswh.df["regionB"] = vbswh.df.eval(
    "presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and abs(deta_jj) <= 4 and hbbjet_msoftdrop >= 150"
)
vbswh.df["regionC"] = vbswh.df.eval(
    "presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and abs(deta_jj) <= 4 and hbbjet_msoftdrop < 150"
)

SIGNAL_REGIONS = ["SR1", "SR2"]

Loading data babies: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.00it/s]


In [13]:
blah = vbswh.sig_df(selection="SR1")

blah[["muon_id_sf", "muon_id_sf_up", "muon_id_sf_dn"]]

,muon_id_sf,muon_id_sf_up,muon_id_sf_dn
0,1.000000,1.000000,1.000000
2,1.000000,1.000000,1.000000
4,0.986421,0.987036,0.985806
5,0.986421,0.987036,0.985806
6,1.000000,1.000000,1.000000
...,...,...,...
37385,0.991008,0.991497,0.990519
37386,0.991008,0.991497,0.990519
37387,0.991008,0.991497,0.990519
37389,0.991008,0.991497,0.990519


# Utilities

In [14]:
class Systematic:
    def __init__(self, name, signal_regions):
        self.name = name
        self.signal_regions = signal_regions
        self.systs = {signal_region: [] for signal_region in signal_regions}
        
    def copy(self, name):
        new_systs = Systematic(name, signal_regions=self.signal_regions)
        new_systs.systs = self.systs
        return new_systs

    def add_syst(self, syst, signal_region):
        self.systs[signal_region].append(syst)
        
    def add_systs(self, systs, signal_region=None):
        if signal_region:
            self.systs[signal_region] += systs
        else:
            for syst_i, syst in enumerate(systs):
                self.systs[self.signal_regions[syst_i]].append(syst)
    
    def get_systs(self):
        return self.systs
    
    def get_systs_str(self, signal_region=None):
        if signal_region:
            systs = self.systs[signal_region]
            if len(systs) == 1:
                return f"{systs[0]:0.1%}"
            else:
                return f"{min(systs):0.1%} - {max(systs):0.1%}"
        else:
            return {SR: self.get_systs_str(signal_region=SR) for SR in self.signal_regions}
        
class SystematicsTable:
    def __init__(self, systs=None, samples=None):
        self.systs = systs or []
        self.samples = samples or []
        
    def add_row(self, syst):
        self.systs.append(syst)
        
    def to_dataframe(self, columns=None):
        rows = []
        for syst in self.systs:
            row = {"Systematic": syst.name}
            row.update(syst.get_systs_str())
            rows.append(row)
        df = pd.DataFrame(rows)
        if columns:
            columns.insert(0, "Systematic")
            return df[columns]
        else:
            return df
        
    def to_csv(self, columns=None, output_csv=None):
        df = self.to_dataframe(columns=columns)
        csv = df.to_csv(index=False)
        if output_csv:
            with open(output_csv, "w") as csv_out:
                csv.write(csv)
        else:
            return csv
        
    def to_latex(self, columns=None, output_tex=None):
        # Convert to Pandas DataFrame
        df = self.to_dataframe(columns=columns)
        
        # Convert to LaTeX
        latex = (df.style
                   .hide(axis="index")
                   .to_latex(column_format="lcc", position="H")
                   .replace("%", "\%"))
        # Insert hlines and centering
        latex = latex.split("\n")
        latex.insert(3, "\\hline")
        latex.insert(2, "\\hline\n\\hline")
        latex.insert(1, "\\begin{center}")
        latex.insert(-3, "\\hline\n\\hline")
        latex.insert(-2, "\\end{center}")
        latex = "\n".join(latex)
        
        if output_tex:
            with open(output_tex, "w") as tex_out:
                tex_out.write(latex)
        else:
            return latex
    
    def to_datacard_json(self, signal_regions=None, output_json=None):
        datacard_systs = {}
        for syst in self.systs:
            labeled_systs = syst.get_systs()
            datacard_systs[syst.name] = [1 + max(systs[SR]) for SR in signal_regions]
            
        if output_json:
            with open(output_json, "w") as json_out:
                json.dump(datacard_systs, json_out)
                
        return datacard_systs
    
def get_year_str(year, doAPV=True):
    if doAPV and year == -2016:
        return "2016preVFP"
    elif doAPV and year == 2016:
        return "2016postVFP"
    else:
        return str(abs(year))

# Scale factors

In [15]:
SIG_SYSTS_TABLE = SystematicsTable(samples=["VBSWH_mkW"])
SIG_SYSTS_LIMIT = SystematicsTable(samples=["VBSWH_mkW"])

In [16]:
def get_systs(sample_name, signal_regions, sf, *sf_variations, year=None):
    global vsbwh
    systs = []
    for SR in signal_regions:
        df = vbswh.sample_df(name=sample_name, selection=SR)
        # Get nominal
        count = df.event_weight.sum()
        # Get delta = nominal - variation for each variation
        deltas = []
        for sf_var in sf_variations:
            if year:
                year_df = df[df.year == year]
                count_var = (
                    np.sum(year_df.event_weight/year_df[sf]*year_df[sf_var])
                    + np.sum(df[df.year != year].event_weight)
                )
            else:
                count_var = np.sum(df.event_weight/df[sf]*df[sf_var])
            deltas.append(abs((count - count_var)/count))
        
        systs.append(max(deltas))
        
    return systs

def get_systs_nonSF(sample_name, signal_regions, signal_regions_up, signal_regions_dn):
    systs = []
    for SR_i, SR in enumerate(signal_regions):
        SR_up = signal_regions_up[SR_i]
        SR_dn = signal_regions_dn[SR_i]
        
        df = vbswh.sample_df(name=sample_name, selection=SR)
        df_up = vbswh.sample_df(name=sample_name, selection=SR_up)
        df_dn = vbswh.sample_df(name=sample_name, selection=SR_dn)
        
        count = df.event_weight.sum()
        count_up = df_up.event_weight.sum()
        count_dn = df_dn.event_weight.sum()
        
        up_perc = abs((count_up - count)/count)
        dn_perc = abs((count - count_dn)/count)
        
        systs.append(max(up_perc, dn_perc))
        
    return systs

### PDF uncertainty

In [17]:
root_files = glob.glob("/ceph/cms/store/user/jguiang/VBSVHSkim/sig_1lep_1ak8_2ak4_pku/VBSWH_mkW_Mjj100toInf_Htobb_dipoleRecoilOn*/merged.root")
gen_sum = 0
pdf_sum = np.zeros(101)
for root_file in root_files:
    with uproot.open(root_file) as f:
        gen_sums = f["Runs"]["genEventSumw"].array(library="np")
        pdf_sums = f["Runs"]["LHEPdfSumw"].array(library="np")
        missed = np.array([len(s) != 101 for s in pdf_sums])
        reshaped = np.vstack(pdf_sums[~missed])
        pdf_sum += np.dot(gen_sums[~missed], reshaped) + np.sum(gen_sums[missed])
        gen_sum += np.sum(gen_sums)
        
pdf_ratio = pdf_sum/gen_sum

with uproot.open(f"../analysis/studies/vbswh/output_{TAG}/Run2/VBSWH_mkW.root") as f:
    df = f.get("pdf_tree").arrays(library="pd")
    
systs = []
for signal_region in SIGNAL_REGIONS:
    SR = df.eval(signal_region)
    count = np.sum(df[SR].event_weight*df[SR].lhe_pdf_0)
    deltas = []
    for i in range(1, 101):
        count_var = np.sum(df[SR].event_weight*df[SR][f"lhe_pdf_{i}"])
        deltas.append(count - count_var/pdf_ratio[i])

    deltas = np.array(deltas)
    systs.append(np.sqrt(np.sum(deltas**2))/count)

pdf_systs = Systematic("PDF variations", SIGNAL_REGIONS)
pdf_systs.add_systs(systs)
SIG_SYSTS_TABLE.add_row(pdf_systs)
SIG_SYSTS_LIMIT.add_row(pdf_systs.copy("CMS_LHE_weights_pdf_vbsvh"))

### $\alpha_S$ uncertainty

In [18]:
# vbswh.df["alphaS_corr_up"] = vbswh.df.alphaS_up
# vbswh.df["alphaS_corr_dn"] = vbswh.df.alphaS_dn

# vbswh.df.loc[vbswh.df.is_signal, "alphaS_corr_up"] = vbswh.df[vbswh.df.is_signal].alphaS_up/pdf_ratio[101]
# vbswh.df.loc[vbswh.df.is_signal, "alphaS_corr_dn"] = vbswh.df[vbswh.df.is_signal].alphaS_up/pdf_ratio[102]

# alphaS_systs = Systematic("$\\alpha_S$ unc.", SIGNAL_REGIONS)
# alphaS_systs.add_systs(
#     get_systs("VBSWH_mkW", SIGNAL_REGIONS, "unity", "alphaS_corr_dn", "alphaS_corr_up")
# )
# SIG_SYSTS_TABLE.add_row(alphaS_systs)
# SIG_SYSTS_LIMIT.add_row(alphaS_systs.copy("alphaS_unc"))

### LHE scale weights

In [19]:
lhe_muR_weights = list(vbswh.df.columns[vbswh.df.columns.str.contains("muF1p0")])
lhe_muF_weights = list(vbswh.df.columns[vbswh.df.columns.str.contains("muR1p0")])

muR_systs = Systematic("$\\mu_R$ scale", SIGNAL_REGIONS)
muR_systs.add_systs(
    get_systs("VBSWH_mkW", SIGNAL_REGIONS, "unity", *lhe_muR_weights)
)

muF_systs = Systematic("$\\mu_F$ scale", SIGNAL_REGIONS)
muF_systs.add_systs(
    get_systs("VBSWH_mkW", SIGNAL_REGIONS, "unity", *lhe_muF_weights)
)

# SIG_SYSTS_TABLE.add_row(muR_systs)                   # muR variations have not effect
SIG_SYSTS_TABLE.add_row(muF_systs)
# SIG_SYSTS_LIMIT.add_row(muR_systs.copy("muR_scale")) # muR variations have not effect
SIG_SYSTS_LIMIT.add_row(muF_systs.copy("CMS_LHE_weights_scale_muF_vbsvh"))

### Parton shower weights

In [20]:
isr_weights = list(vbswh.df.columns[vbswh.df.columns.str.contains("fsr1p0")])
fsr_weights = list(vbswh.df.columns[vbswh.df.columns.str.contains("isr1p0")])

isr_sf_systs = Systematic("Parton shower ISR weights", SIGNAL_REGIONS)
isr_sf_systs.add_systs(
    get_systs("VBSWH_mkW", SIGNAL_REGIONS, "unity", *isr_weights)
)

fsr_sf_systs = Systematic("Parton shower FSR weights", SIGNAL_REGIONS)
fsr_sf_systs.add_systs(
    get_systs("VBSWH_mkW", SIGNAL_REGIONS, "unity", *fsr_weights)
)

SIG_SYSTS_TABLE.add_row(isr_sf_systs)
SIG_SYSTS_TABLE.add_row(fsr_sf_systs)
SIG_SYSTS_LIMIT.add_row(isr_sf_systs.copy("CMS_PSWeight_ISR_vbsvh"))
SIG_SYSTS_LIMIT.add_row(fsr_sf_systs.copy("CMS_PSWeight_FSR_vbsvh"))

### Pileup reweighting

In [21]:
pu_sf_systs = Systematic("Pileup reweighting", SIGNAL_REGIONS)
pu_sf_systs.add_systs(
    get_systs("VBSWH_mkW", SIGNAL_REGIONS, "pu_sf", "pu_sf_dn", "pu_sf_up")
)
SIG_SYSTS_TABLE.add_row(pu_sf_systs)
SIG_SYSTS_LIMIT.add_row(pu_sf_systs.copy("CMS_vbswhboosted_puWeight"))

### Pileup jet ID scale factors

In [22]:
puid_sf_systs = Systematic("Pileup jet ID", SIGNAL_REGIONS)
puid_sf_systs.add_systs(
    get_systs("VBSWH_mkW", SIGNAL_REGIONS, "puid_sf", "puid_sf_dn", "puid_sf_up")
)
SIG_SYSTS_TABLE.add_row(puid_sf_systs)
SIG_SYSTS_LIMIT.add_row(puid_sf_systs.copy("CMS_vbswhboosted_puJetId"))

### L1 Prefiring weight

In [23]:
prefire_sf_systs = Systematic("L1 pre-fire corrections", SIGNAL_REGIONS)
prefire_sf_systs.add_systs(
    get_systs("VBSWH_mkW", SIGNAL_REGIONS, "prefire_sf", "prefire_sf_up", "prefire_sf_dn")
)
SIG_SYSTS_TABLE.add_row(prefire_sf_systs)
SIG_SYSTS_LIMIT.add_row(prefire_sf_systs.copy("CMS_PrefireWeight_13TeV"))

### HLT scale factors

In [24]:
# trig_sf_systs = Systematic("HLT scale factors", SIGNAL_REGIONS)
# trig_sf_systs.add_systs(
#     get_systs("VBSWH_mkW", SIGNAL_REGIONS, "trig_sf", "trig_sf_up", "trig_sf_dn")
# )
# SIG_SYSTS_TABLE.add_row(trig_sf_systs)
# SIG_SYSTS_LIMIT.add_row(trig_sf_systs.copy("hlt_sfs"))

trig_elec_sf_systs = Systematic("Single-electron HLT scale factors", SIGNAL_REGIONS)
trig_elec_sf_systs.add_systs(
    get_systs("VBSWH_mkW", SIGNAL_REGIONS, "trig_elec_sf", "trig_elec_sf_up", "trig_elec_sf_dn")
)
SIG_SYSTS_TABLE.add_row(trig_elec_sf_systs)
SIG_SYSTS_LIMIT.add_row(trig_elec_sf_systs.copy("CMS_vbswhboosted_hlt_e_13TeV"))

trig_muon_sf_systs = Systematic("Single-muon HLT scale factors", SIGNAL_REGIONS)
trig_muon_sf_systs.add_systs(
    get_systs("VBSWH_mkW", SIGNAL_REGIONS, "trig_muon_sf", "trig_muon_sf_up", "trig_muon_sf_dn")
)
SIG_SYSTS_TABLE.add_row(trig_muon_sf_systs)
SIG_SYSTS_LIMIT.add_row(trig_muon_sf_systs.copy("CMS_vbswhboosted_hlt_m_13TeV"))

### Statistical uncertainty

In [25]:
stat_systs = Systematic("Simulation stat. unc.", SIGNAL_REGIONS)
stat_systs.add_systs(
    [
        vbswh.sig_error(selection="SR1")/vbswh.sig_count(selection="SR1"), 
        vbswh.sig_error(selection="SR2")/vbswh.sig_count(selection="SR2")
    ]
)
SIG_SYSTS_TABLE.add_row(stat_systs)
SIG_SYSTS_LIMIT.add_row(stat_systs.copy("CMS_vbswhboosted_mcstat"))

### Lepton scale factors

In [26]:
# lep_sf_systs = Systematic("Lepton scale factors", SIGNAL_REGIONS)
# lep_sf_systs.add_systs(
#     get_systs("VBSWH_mkW", SIGNAL_REGIONS, "lep_id_sf", "lep_id_sf_up", "lep_id_sf_dn")
# )
# lep_sf_systs.add_systs(
#     get_systs("VBSWH_mkW", SIGNAL_REGIONS, "elec_reco_sf", "elec_reco_sf_up", "elec_reco_sf_dn")
# )
# lep_sf_systs.add_systs(
#     get_systs("VBSWH_mkW", SIGNAL_REGIONS, "muon_iso_sf", "muon_iso_sf_up", "muon_iso_sf_dn")
# )
# SIG_SYSTS_TABLE.add_row(lep_sf_systs)

# lep_id_sf_systs = Systematic("lep_id", SIGNAL_REGIONS)
# lep_id_sf_systs.add_systs(
#     get_systs("VBSWH_mkW", SIGNAL_REGIONS, "lep_id_sf", "lep_id_sf_up", "lep_id_sf_dn")
# )
# SIG_SYSTS_LIMIT.add_row(lep_id_sf_systs)

elec_id_sf_systs = Systematic("Electron ID scale factors", SIGNAL_REGIONS)
elec_id_sf_systs.add_systs(
    get_systs("VBSWH_mkW", SIGNAL_REGIONS, "elec_id_sf", "elec_id_sf_up", "elec_id_sf_dn")
)
SIG_SYSTS_TABLE.add_row(elec_id_sf_systs)
SIG_SYSTS_LIMIT.add_row(elec_id_sf_systs.copy("CMS_vbswhboosted_id_e_13TeV"))

muon_id_sf_systs = Systematic("Muon ID scale factors", SIGNAL_REGIONS)
muon_id_sf_systs.add_systs(
    get_systs("VBSWH_mkW", SIGNAL_REGIONS, "muon_id_sf", "muon_id_sf_up", "muon_id_sf_dn")
)
SIG_SYSTS_TABLE.add_row(muon_id_sf_systs)
SIG_SYSTS_LIMIT.add_row(muon_id_sf_systs.copy("CMS_vbswhboosted_id_m_13TeV"))

elec_reco_sf_systs = Systematic("Electron reco. scale factors", SIGNAL_REGIONS)
elec_reco_sf_systs.add_systs(
    get_systs("VBSWH_mkW", SIGNAL_REGIONS, "elec_reco_sf", "elec_reco_sf_up", "elec_reco_sf_dn")
)
SIG_SYSTS_TABLE.add_row(elec_reco_sf_systs)
SIG_SYSTS_LIMIT.add_row(elec_reco_sf_systs.copy("CMS_vbswhboosted_reco_e_13TeV"))

muon_iso_sf_systs = Systematic("Muon iso. scale factors", SIGNAL_REGIONS)
muon_iso_sf_systs.add_systs(
    get_systs("VBSWH_mkW", SIGNAL_REGIONS, "muon_iso_sf", "muon_iso_sf_up", "muon_iso_sf_dn")
)
SIG_SYSTS_TABLE.add_row(muon_iso_sf_systs)
SIG_SYSTS_LIMIT.add_row(muon_iso_sf_systs.copy("CMS_vbswhboosted_iso_m_13TeV"))

### ParticleNet Xbb scale factors

In [27]:
xbb_sf_systs = Systematic("ParticleNet Xbb scale factors", SIGNAL_REGIONS)
# xbb_sf_systs.add_systs(
#     get_systs("VBSWH_mkW", SIGNAL_REGIONS, "xbb_sf", "xbb_sf_dn", "xbb_sf_up")
# )
# SIG_SYSTS_TABLE.add_row(xbb_sf_systs)
# SIG_SYSTS_LIMIT.add_row(xbb_sf_systs.copy("xbb_sfs"))

for year in [-2016, 2016, 2017, 2018]:
    stupid_cms_year_str = get_year_str(year).replace("20", "")
    xbb_sf_systs_ = Systematic(f"CMS_vbswhboosted_bTagWeightXbb_13TeV_{stupid_cms_year_str}", SIGNAL_REGIONS)
    xbb_sf_systs_.add_systs(
        get_systs("VBSWH_mkW", SIGNAL_REGIONS, "xbb_sf", "xbb_sf_dn", "xbb_sf_up", year=year)
    )
    xbb_sf_systs.add_systs(
        get_systs("VBSWH_mkW", SIGNAL_REGIONS, "xbb_sf", "xbb_sf_dn", "xbb_sf_up", year=year)
    )
    SIG_SYSTS_LIMIT.add_row(xbb_sf_systs_)
    
SIG_SYSTS_TABLE.add_row(xbb_sf_systs)

### DeepJet b-tagging scale factors

In [28]:
# btag_sf_systs = Systematic("DeepJet b-tagging scale factors", SIGNAL_REGIONS)
# btag_sf_systs.add_systs(
#     get_systs("VBSWH_mkW", SIGNAL_REGIONS, "btag_sf", "btag_sf_dn", "btag_sf_up")
# )
# SIG_SYSTS_TABLE.add_row(btag_sf_systs)
# SIG_SYSTS_LIMIT.add_row(btag_sf_systs.copy("CMS_bTagWeightDeepJet_13TeV"))

btag_sf_systs = Systematic("DeepJet b-tagging scale factors (b/c jets)", SIGNAL_REGIONS)
btag_sf_systs.add_systs(
    get_systs("VBSWH_mkW", SIGNAL_REGIONS, "bc_btag_sf", "bc_btag_sf_dn", "bc_btag_sf_up")
)
SIG_SYSTS_TABLE.add_row(btag_sf_systs)
SIG_SYSTS_LIMIT.add_row(btag_sf_systs.copy("CMS_bTagWeightDeepJet_bcJets_13TeV"))

print("b jets")
print(f"{btag_sf_systs.get_systs()['SR1'][0]*100:.6f}")

print("c jets")
print(f"{btag_sf_systs.get_systs()['SR1'][0]*100:.6f}")

btag_sf_systs = Systematic("DeepJet b-tagging scale factors (light jets)", SIGNAL_REGIONS)
btag_sf_systs.add_systs(
    get_systs("VBSWH_mkW", SIGNAL_REGIONS, "light_btag_sf", "light_btag_sf_dn", "light_btag_sf_up")
)
SIG_SYSTS_TABLE.add_row(btag_sf_systs)
SIG_SYSTS_LIMIT.add_row(btag_sf_systs.copy("CMS_bTagWeightDeepJet_lightJets_13TeV"))

print("light jets")
print(f"{btag_sf_systs.get_systs()['SR1'][0]*100:.6f}")

b jets
0.021990
c jets
0.021990
light jets
0.152389


### MET uncertainty

In [29]:
met_unc_systs = Systematic("MET unc.", SIGNAL_REGIONS)
met_unc_systs.add_systs(
    get_systs_nonSF("VBSWH_mkW", SIGNAL_REGIONS, ["SR1_dn", "SR2_dn"], ["SR1_up", "SR2_up"])
)
SIG_SYSTS_TABLE.add_row(met_unc_systs)
SIG_SYSTS_LIMIT.add_row(met_unc_systs.copy("CMS_metUncl_13TeV"))

# Jet energy

In [30]:
def get_jet_energy_systs(nominal_cflow, up_cflow, dn_cflow, signal_regions, name):

    syst_up_cflow = (up_cflow - nominal_cflow)/nominal_cflow
    syst_dn_cflow = (nominal_cflow - dn_cflow)/nominal_cflow

    systs = Systematic(name, signal_regions.keys())
    for SR, cut_name in signal_regions.items():
        systs.add_syst(
            max(
                syst_up_cflow[cut_name].n_pass_weighted,
                syst_dn_cflow[cut_name].n_pass_weighted
            ),
            signal_region=SR
        )
        
    return systs

### Jet energy corrections

In [31]:
jec_systs = Systematic("Jet energy scale", SIGNAL_REGIONS)
cflow = "VBSWH_mkW_Mjj100toInf_Htobb_dipoleRecoilOn_Cutflow.cflow"
for year in ["2016preVFP", "2016postVFP", "2017", "2018"]:
    nominal_cutflow = Cutflow.from_file(f"../analysis/studies/vbswh/output_{TAG}/Run2/VBSWH_mkW_cutflow.cflow")
    up_cutflow = Cutflow.from_file(f"../analysis/studies/vbswh/output_{TAG}_jec_up/{year}/{cflow}")
    dn_cutflow = Cutflow.from_file(f"../analysis/studies/vbswh/output_{TAG}_jec_dn/{year}/{cflow}")
    for nom_year in ["2016preVFP", "2016postVFP", "2017", "2018"]:
        if year != nom_year:
            up_cutflow += Cutflow.from_file(f"../analysis/studies/vbswh/output_{TAG}/{nom_year}/{cflow}")
            dn_cutflow += Cutflow.from_file(f"../analysis/studies/vbswh/output_{TAG}/{nom_year}/{cflow}")
            
    jec_systs_ = get_jet_energy_systs(
        nominal_cutflow,
        up_cutflow,
        dn_cutflow,
        {"SR1": "XbbGt0p9_MSDLt150", "SR2": "STGt1500"},
        f"Jet energy scale"
    )
    jec_systs.add_systs([s[0] for s in jec_systs_.systs.values()])
    SIG_SYSTS_LIMIT.add_row(jec_systs_.copy(f"CMS_scale_j_13TeV_{year[2:]}"))
    
SIG_SYSTS_TABLE.add_row(jec_systs)

### Jet energy resolution

In [32]:
jer_systs = Systematic("Jet energy resolution", SIGNAL_REGIONS)
for year in ["2016preVFP", "2016postVFP", "2017", "2018"]:
    nominal_cutflow = Cutflow.from_file(f"../analysis/studies/vbswh/output_{TAG}/Run2/VBSWH_mkW_cutflow.cflow")
    up_cutflow = Cutflow.from_file(f"../analysis/studies/vbswh/output_{TAG}_jer_up/{year}/{cflow}")
    dn_cutflow = Cutflow.from_file(f"../analysis/studies/vbswh/output_{TAG}_jer_dn/{year}/{cflow}")
    for nom_year in ["2016preVFP", "2016postVFP", "2017", "2018"]:
        if year != nom_year:
            up_cutflow += Cutflow.from_file(f"../analysis/studies/vbswh/output_{TAG}/{nom_year}/{cflow}")
            dn_cutflow += Cutflow.from_file(f"../analysis/studies/vbswh/output_{TAG}/{nom_year}/{cflow}")
            
    jer_systs_ = get_jet_energy_systs(
        nominal_cutflow,
        up_cutflow,
        dn_cutflow,
        {"SR1": "XbbGt0p9_MSDLt150", "SR2": "STGt1500"},
        "Jet energy resolution"
    )
    jer_systs.add_systs([s[0] for s in jer_systs_.systs.values()])
    SIG_SYSTS_LIMIT.add_row(jer_systs_.copy(f"CMS_res_j_13TeV_{year[2:]}"))
    
SIG_SYSTS_TABLE.add_row(jer_systs)

# Other

### Luminosity

In [33]:
lumi_systs = Systematic("Luminosity", SIGNAL_REGIONS)
lumi_systs.add_systs([0.016, 0.016])
SIG_SYSTS_TABLE.add_row(lumi_systs)
SIG_SYSTS_LIMIT.add_row(lumi_systs.copy("lumi_13TeV_correlated"))

### H to bb BR uncertainty

In [34]:
hbb_br_systs = Systematic("\\Htobb BR", SIGNAL_REGIONS)
hbb_br_systs.add_systs([0.0127, 0.0127])
SIG_SYSTS_TABLE.add_row(hbb_br_systs)
SIG_SYSTS_LIMIT.add_row(hbb_br_systs.copy("BR_hbb"))

In [35]:
SIG_SYSTS_TABLE.to_dataframe(columns=["SR1"])

,Systematic,SR1
0,PDF variations,1.8%
1,$\mu_F$ scale,17.5%
2,Parton shower ISR weights,0.6%
3,Parton shower FSR weights,1.7%
4,Pileup reweighting,0.2%
5,Pileup jet ID,0.8%
6,L1 pre-fire corrections,0.9%
7,Single-electron HLT scale factors,0.7%
8,Single-muon HLT scale factors,0.1%
9,Simulation stat. unc.,0.8%


In [36]:
class Datacard:
    def __init__(self, obs, sig, bkg, systs):
        self.obs = obs
        self.n_obs = len(obs)
        
        self.sig_labels = []
        self.sig_yields = [[] for _ in range(self.n_obs)]
        for sig_label, sig_yields in sig.items():
            self.sig_labels.append(sig_label)
            for bin_i, sig_yield in enumerate(sig_yields):
                self.sig_yields[bin_i].append(sig_yield)
                
        self.bkg_labels = []
        self.bkg_yields = [[] for _ in range(self.n_obs)]
        for bkg_label, bkg_yields in bkg.items():
            self.bkg_labels.append(bkg_label)
            for bin_i, bkg_yield in enumerate(bkg_yields):
                self.bkg_yields[bin_i].append(bkg_yield)

        self.n_sig = len(self.sig_labels)
        self.n_bkg = len(self.bkg_labels)
        
        self.column_width = max([len(l) for l in self.sig_labels + self.bkg_labels])+2
        self.column_width = max(self.column_width, 12)
                
        self.syst_labels = []
        self.systs = []
        n_samples = self.n_sig + self.n_bkg
        w = self.column_width
        for sample_label, labeled_systs in systs.items():
            for syst_label, syst_values in labeled_systs.items():
                # Register syst
                if syst_label not in self.syst_labels:
                    self.syst_labels.append(syst_label)
                    self.systs.append([f"{'-':>{w}}" for _ in range(self.n_obs*(n_samples))])
                
                # Get index of syst label
                label_i = self.syst_labels.index(syst_label)
                
                # Assign syst values
                if sample_label in self.sig_labels:
                    for value_i, syst_value in enumerate(syst_values):
                        syst_i = self.sig_labels.index(sample_label)+value_i*n_samples
                        self.systs[label_i][syst_i] = f"{syst_value:{w}.4f}" 
                elif sample_label in self.bkg_labels:
                    for value_i, syst_value in enumerate(syst_values):
                        syst_i = self.n_sig+self.bkg_labels.index(sample_label)+value_i*n_samples
                        self.systs[label_i][syst_i] = f"{syst_value:{w}.4f}"  
                else:
                    raise Exception(f"{sample_label} not found")
                    
        self.header_width = max([len(l) for l in self.syst_labels])+2
        self.header_width = max(self.header_width, 12)
        
        self.content = None
        self.__create()
        
    def __create(self):
        cw = self.column_width
        hw = self.header_width
        hline = "-"*(hw+5 + cw*(self.n_sig + self.n_bkg)*self.n_obs) + "\n"
        content = ""
        content += f"imax {self.n_obs} number of channels\n"
        content += f"jmax {self.n_bkg} number of backgrounds\n"
        content += f"kmax {len(self.systs)} number of nuisance parameters\n"
        content += hline
        content += f"{'bin':<{hw+5}}"
        content +=  "".join([f"{'bin'+str(i+1):>{cw}}" for i in range(self.n_obs)])
        content +=  "\n"
        content += f"{'observation':<{hw+5}}"
        content +=  "".join([f"{n:{cw}d}" for n in self.obs])
        content +=  "\n"
        content += hline
        content += f"{'bin':<{hw+5}}"
        content += "".join([f"{'bin'+str(i+1):>{cw}}" for i in range(self.n_obs) for _ in range(self.n_sig + self.n_bkg)])
        content +=  "\n"
        content += f"{'process':<{hw+5}}"
        content +=  "".join([f"{l:>{cw}}" for _ in range(self.n_obs) for l in self.sig_labels + self.bkg_labels])
        content +=  "\n"
        content += f"{'process':<{hw+5}}"
        content += "".join([f"{i:>{cw}}" for _ in range(self.n_obs) for i in range(self.n_sig + self.n_bkg)])
        content +=  "\n"
        content += f"{'rate':<{hw+5}}"
        content +=  "".join([f"{y[i]:{cw}.2f}" for y in self.sig_yields for i in range(self.n_sig)])
        content +=  "".join([f"{y[i]:{cw}.2f}" for y in self.bkg_yields for i in range(self.n_bkg)])
        content +=  "\n"
        content += hline
        for syst_i, syst_values in enumerate(self.systs):
            content += f"{self.syst_labels[syst_i]:<{hw}}"
            content +=  " lnN "
            content +=  "".join(syst_values)
            content += "\n"
            
        self.content = content
        
    def write(self, output_dat):
        with open(output_dat, "w") as dat_out:
            dat_out.write(self.content)

In [37]:
with open("AN_numbers.json", "r") as f_in:
    AN_numbers = json.load(f_in)

datacard_systs = {
    "TotalBkg": {
        "CMS_vbswhboosted_abcd_syst": [1 + AN_numbers["BkgEstTotalSystErr"]/100],
        "CMS_vbswhboosted_abcd_stat": [1 + AN_numbers["BkgEstStatErr"]/100]
    }
}

total_sig_syst = 0
datacard_systs["VBSWH_mkW"] = {}
for syst_obj in SIG_SYSTS_LIMIT.systs:
    systs = syst_obj.get_systs()
    datacard_systs["VBSWH_mkW"][syst_obj.name] = [1 + systs["SR1"][0]]
    total_sig_syst += systs["SR1"][0]**2
    
total_sig_syst = np.sqrt(total_sig_syst)
    
pred_bkg = AN_numbers["PredBkg"]

datacard = Datacard(
    [round(pred_bkg)],
    {"VBSWH_mkW": [vbswh.sig_count(selection="SR1")]},
    {"TotalBkg": [pred_bkg]},
    datacard_systs
)
# datacard.write("../combine/vbswh/datacards/vbswh.dat")
# print("Wrote blinded datacard")

datacard_unblinded = Datacard(
    [vbswh.data_count(selection="SR1")],
    {"VBSWH_mkW": [vbswh.sig_count(selection="SR1")]},
    {"TotalBkg": [pred_bkg]},
    datacard_systs
)
# datacard_unblinded.write("../combine/vbswh/datacards/vbswh_unblinded.dat")
# print("Wrote unblinded datacard")

print("")
if BLIND:
    print(datacard.content)
else:
    print(datacard_unblinded.content)


imax 1 number of channels
jmax 1 number of backgrounds
kmax 33 number of nuisance parameters
-----------------------------------------------------------------------------
bin                                                          bin1
observation                                                   130
-----------------------------------------------------------------------------
bin                                                          bin1        bin1
process                                                 VBSWH_mkW    TotalBkg
process                                                         0           1
rate                                                       366.39      108.00
-----------------------------------------------------------------------------
CMS_vbswhboosted_abcd_syst                       lnN            -      1.1280
CMS_vbswhboosted_abcd_stat                       lnN            -      1.1340
CMS_LHE_weights_pdf_vbsvh                        lnN       1.0181       

In [ ]:
print(SIG_SYSTS_TABLE.to_latex(columns=["SR1"]))

In [ ]:
print(SIG_SYSTS_TABLE.to_csv(columns=["SR1"]))

In [ ]:
total_SR1_sig_syst = 0
total_SR2_sig_syst = 0
for syst_obj in SIG_SYSTS_LIMIT.systs:
    systs = syst_obj.get_systs()
    total_SR1_sig_syst += systs["SR1"][0]**2
    total_SR2_sig_syst += systs["SR2"][0]**2
    
total_SR1_sig_syst = np.sqrt(total_SR1_sig_syst)
total_SR2_sig_syst = np.sqrt(total_SR2_sig_syst)

with open("AN_numbers.json", "w") as f_out:
    AN_numbers["ExpSigSystErr"] = round(total_SR1_sig_syst*AN_numbers["ExpSig"], 1)
    AN_numbers["SRTwoExpSigSystErr"] = round(total_SR2_sig_syst*AN_numbers["SRTwoExpSig"], 1)
    AN_numbers["Obs"] = vbswh.data_count(selection="SR1")
    json.dump(AN_numbers, f_out)

In [ ]:
for key, value in AN_numbers.items():
    print(f"\\newcommand{{\\{key}}}{{{value}}}")

In [32]:
print(SIG_SYSTS_TABLE.to_latex(columns=["SR2"]))

\begin{table}[H]
\begin{center}
\begin{tabular}{lcc}
\hline
\hline
Systematic & SR2 \\
\hline
PDF variations & 4.3\% \\
$\mu_F$ scale & 20.8\% \\
Parton shower ISR weights & 0.5\% \\
Parton shower FSR weights & 1.7\% \\
Pileup reweighting & 0.3\% \\
Pileup jet ID & 0.9\% \\
L1 pre-fire corrections & 0.9\% \\
Single-electron HLT scale factors & 0.7\% \\
Single-muon HLT scale factors & 0.1\% \\
Simulation stat. unc. & 1.5\% \\
Electron ID scale factors & 1.1\% \\
Muon ID scale factors & 0.1\% \\
Electron reco. scale factors & 0.3\% \\
Muon iso. scale factors & 0.0\% \\
ParticleNet Xbb scale factors & 0.5\% - 1.2\% \\
DeepJet b-tagging scale factors (b/c jets) & 0.0\% \\
DeepJet b-tagging scale factors (light jets) & 0.1\% \\
MET unc. & 0.3\% \\
Jet energy scale & 0.7\% - 3.7\% \\
Jet energy resolution & 0.0\% - 0.2\% \\
Luminosity & 1.6\% \\
\Htobb BR & 1.3\% \\
\hline
\hline
\end{tabular}
\end{center}
\end{table}



In [35]:
# "Datacard" for SR2 (NOT USED)
# datacard_systs = {
#     "TotalBkg": {
#         "est_syst": [1.34],
#         "est_stat": [1.13]
#     }
# }

# datacard_systs["VBSWH_mkW"] = {}
# for syst_obj in SIG_SYSTS_LIMIT.systs:
#     systs = syst_obj.get_systs()
#     datacard_systs["VBSWH_mkW"][syst_obj.name] = [1 + systs["SR2"][0]]
    
# pred_bkg = vbswh.data_count(selection="regionA")/vbswh.data_count(selection="regionB")*vbswh.data_count(selection="regionC")
# datacard = Datacard(
#     [int(pred_bkg)],
#     {"VBSWH_mkW": [vbswh.sig_count(selection="SR2")]},
#     {"TotalBkg": [pred_bkg]},
#     datacard_systs
# )
    
# print(datacard.content)